In [1]:
%matplotlib inline

In [2]:
import torch
import os
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import finalprojectneuralnetworks as fpnn
import csv
import time

In [3]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"
fp_csv_file = "EEG_Eye_State_Train.csv"
# fp_csv_file = "Dummy_Train.csv"
eeg_data = pd.read_csv(
    os.path.join(root_directory, fp_csv_file), delimiter=",", usecols=range(15)
).dropna(axis=0)

print(eeg_data.iloc[0, :])
open_or_closed = eeg_data.iloc[0, 14]
channel_data = eeg_data.iloc[0, 0:14]
channel_data = np.array([channel_data])
channel_data = channel_data.astype("float").reshape(-1, 14)
print(open_or_closed)
print(channel_data)
eeg_data.shape

AF3             4309.74
F7              4052.82
F3              4285.13
FC5             4155.38
T7              4369.23
P7              4646.67
O1              4100.00
O2              4649.74
P8              4221.54
T8              4230.26
FC6             4206.15
F4              4292.82
F8              4598.97
AF4             4362.56
eyeDetection       0.00
Name: 0, dtype: float64
0
[[4309.74 4052.82 4285.13 4155.38 4369.23 4646.67 4100.   4649.74 4221.54
  4230.26 4206.15 4292.82 4598.97 4362.56]]


(11984, 15)

In [4]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

train_file = "EEG_Eye_State_Train.csv"
validate_file = "EEG_Eye_State_Validate.csv"
test_file = "EEG_Eye_State_Test.csv"

# train_file = "Dummy_Train.csv"
# validate_file = "Dummy_Validate.csv"
# test_file = "Dummy_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
print(train_dataset.__len__())
train_dataset.__getitem__(0)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6             float64
F4              float64
F8              float64
AF4             float64
eyeDetection      int64
dtype: object
AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6             float64
F4              float64
F8              float64
AF4             float64
eyeDetection      int64
dtype: object
AF3             float64
F7              float64
F3              float64
FC5             float64
T7              float64
P7              float64
O1              float64
O2              float64
P8              float64
T8              float64
FC6         

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [6]:
# class ConvNet(torch.nn.Module):
#     def __init__(
#         self,
#         n_inputs,
#         n_hiddens_per_conv_layer,
#         n_hiddens_per_fc_layer,
#         n_outputs,
#         patch_size_per_conv_layer,
#         stride_per_conv_layer,
#         activation_function="tanh",
#         device="cpu",
#     ):

#         super().__init__()

#         self.device = device

#         n_conv_layers = len(n_hiddens_per_conv_layer)
#         if (
#             len(patch_size_per_conv_layer) != n_conv_layers
#             or len(stride_per_conv_layer) != n_conv_layers
#         ):
#             raise Exception(
#                 "The lengths of n_hiddens_per_conv_layer, patch_size_per_conv_layer, and stride_per_conv_layer must be equal."
#             )

#         self.activation_function = (
#             torch.tanh if activation_function == "tanh" else torch.relu
#         )

#         # Create all convolutional layers
#         # First argument to first Conv1d is number of channels for a sample
#         n_in = 14
#         input_hw = int(
#             np.sqrt(n_inputs)
#         )  # original input image height (=width because image assumed square)
#         self.conv_layers = torch.nn.ModuleList()
#         for nh, patch_size, stride in zip(
#             n_hiddens_per_conv_layer, patch_size_per_conv_layer, stride_per_conv_layer
#         ):
#             self.conv_layers.append(
#                 torch.nn.Conv1d(n_in, nh, kernel_size=patch_size, stride=stride)
#             )
#             conv_layer_output_hw = (input_hw - patch_size) // stride + 1
#             input_hw = conv_layer_output_hw  # for next trip through this loop
#             n_in = nh

#         # Create all fully connected layers.  First must determine number of inputs to first
#         # fully-connected layer that results from flattening the images coming out of the last
#         # convolutional layer.
#         n_in = input_hw ** 2 * n_in  # n_hiddens_per_fc_layer[0]
#         self.fc_layers = torch.nn.ModuleList()
#         for nh in n_hiddens_per_fc_layer:
#             self.fc_layers.append(torch.nn.Linear(n_in, nh))
#             n_in = nh
#         self.fc_layers.append(torch.nn.Linear(n_in, n_outputs))

#         self.to(self.device)

#     def forward_all_outputs(self, X):
#         n_samples = X.shape[0]
#         Ys = [X]
#         for conv_layer in self.conv_layers:
#             Ys.append(self.activation_function(conv_layer(Ys[-1])))

#         for layeri, fc_layer in enumerate(self.fc_layers[:-1]):
#             if layeri == 0:
#                 Ys.append(
#                     self.activation_function(fc_layer(Ys[-1].reshape(n_samples, -1)))
#                 )
#             else:
#                 Ys.append(self.activation_function(fc_layer(Ys[-1])))

#         Ys.append(self.fc_layers[-1](Ys[-1]))
#         return Ys

#     def forward(self, X):
#         Ys = self.forward_all_outputs(X)
#         return Ys[-1]

#     def train(
#         self, X, T, batch_size, n_epochs, learning_rate, method="sgd", verbose=True
#     ):

#         # Set data matrices to torch.tensors if not already.
#         if not isinstance(X, torch.Tensor):
#             X = torch.from_numpy(X).float().to(self.device)
#         if not isinstance(T, torch.Tensor):
#             T = (
#                 torch.from_numpy(T).long().to(self.device)
#             )  # required for classification in pytorch

#         X.requires_grad_(True)

#         self.classes = torch.unique(T)

#         if method == "sgd":
#             optimizer = torch.optim.SGD(
#                 self.parameters(), lr=learning_rate, momentum=0.9
#             )
#         else:
#             optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

#         CELoss = torch.nn.CrossEntropyLoss(reduction="mean")
#         self.error_trace = []

#         for epoch in range(n_epochs):

#             num_batches = X.shape[0] // batch_size
#             loss_sum = 0

#             for k in range(num_batches):
#                 start = k * batch_size
#                 end = (k + 1) * batch_size
#                 X_batch = X[start:end, ...]
#                 T_batch = T[start:end, ...]

#                 Y = self.forward(X_batch)

#                 loss = CELoss(Y, T_batch)
#                 loss.backward()

#                 # Update parameters
#                 optimizer.step()
#                 optimizer.zero_grad()

#                 loss_sum += loss

#             self.error_trace.append(loss_sum / num_batches)

#             if n_epochs >= 10:
#                 if verbose and (epoch + 1) % (n_epochs // 10) == 0:
#                     print(
#                         f"{method}: Epoch {epoch + 1} Loss {self.error_trace[-1]:.3f}"
#                     )
#             else:
#                 print(f"{method}: Epoch {epoch + 1} Loss {self.error_trace[-1]:.3f}")

#         return self

#     def softmax(self, Y):
#         """Apply to final layer weighted sum outputs"""
#         # Trick to avoid overflow
#         maxY = torch.max(Y, axis=1)[0].reshape((-1, 1))
#         expY = torch.exp(Y - maxY)
#         denom = torch.sum(expY, axis=1).reshape((-1, 1))
#         Y = expY / denom
#         return Y

#     def use(self, X):
#         # Set input matrix to torch.tensors if not already.
#         if not isinstance(X, torch.Tensor):
#             X = torch.from_numpy(X).float().to(self.device)
#         Y = self.forward(X)
#         probs = self.softmax(Y)
#         classes = self.classes[torch.argmax(probs, axis=1).cpu().numpy()]
#         return classes.cpu().numpy(), probs.detach().cpu().numpy()

In [7]:
model = fpnn.CNN(
    14,
    [10, 20],
    [5, 5],
    2,
    patch_size_per_conv_layer=[5, 7],
    stride_per_conv_layer=[2, 2],
).to(device)
print(model)

i == 0
i == 0
CNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [8]:
from functools import reduce
from operator import mul

X = torch.rand(14, 100, device=device)
print(X.shape)
T = torch.randint(0, 2, (100, 1), device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([14, 100])
Conv1d(14, 10, kernel_size=(5,), stride=(2,))
Conv1d(10, 20, kernel_size=(7,), stride=(2,))


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x21 and 20x5)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    # first = True
    for batch, (X, T) in enumerate(dataloader):
        X, T = X.to(device), T.to(device)

        # Compute prediction error
        Y = model(X.float())
        # if first:
        #     first = False
        #     print('Y.shape', Y.shape)
        #     print('T.shape', T.shape)
        #     print('T: ', T[0])
        loss = loss_fn(Y, T.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


#         if batch % 50 == 0:
#             loss, current = loss.item(), batch * len(X)
# print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, T in dataloader:
            X, T = X.to(device), T.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, T.long()).item()
            correct += (pred.argmax(1) == T).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [ ]:
def train_for_epochs(epochs, train_dataloader, model, loss_fn, optimizer):
    start = time.time()
    for t in range(epochs):
        train(train_dataloader, model, loss_fn, optimizer)
        if t % 100 == 0:
            print(f"Epoch {t}\n-------------------------------")
            test(validate_dataloader, model, loss_fn)
    end = time.time()
    print("Done: ", end - start, " seconds")

In [ ]:
# fashion_training_data = datasets.FashionMNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
# )
# fashion_train_dataloader = DataLoader(fashion_training_data, batch_size=64)
# n_samples, n_inputs = train_dataset.__len__(), 28*28
# n_outputs = 1
# n_hiddens = [10, 10]
# model = fpnn.NeuralNetwork(28*28, n_hiddens, 10).to(device)
# print(model)

# learning_rate = 0.5 / (n_samples * n_outputs)  ## Larger learning rate

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss_fn = torch.nn.CrossEntropyLoss()

# train(fashion_train_dataloader, model, loss_fn, optimizer)

In [ ]:
for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

In [ ]:
n_samples, n_inputs = train_dataset.__len__(), 14
n_outputs = 2
n_hiddens = [5, 5]
model = fpnn.NeuralNetwork(n_inputs, n_hiddens, n_outputs).to(device)
print(model)

# learning_rate = 0.05 / (n_samples * n_outputs)
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 10
train_for_epochs(epochs, train_dataloader, model, loss_fn, optimizer)